---



# Unit 2 - Part 2a: The Anatomy of a Prompt

## 1. Introduction: Stochasticity (Randomness)

Why does the AI give different answers? Because it is **Stochastic** (Random).

It predicts the NEXT TOKEN based on probability.

### Visualizing the Prediction
Input: `"The sky is..."`

| Word | Probability | Selected? (Temp=0) | Selected? (Temp=1) |
|------|-------------|--------------------|--------------------|
| Blue | 80% | ✅ | ❌ |
| Gray | 15% | ❌ | ✅ |
| Green| 1% | ❌ | ❌ |

Prompt Engineering is the art of **manipulating these probabilities**.

In [3]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.6 MB/s eta 0:00:00


In [5]:
# Setup
from dotenv import load_dotenv
load_dotenv()
from google.colab import userdata
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini')

# Using Low Temp for consistent comparison
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

## 2. The CO-STAR Framework (simplified)

A good prompt usually has:
1.  **C**ontext (Who are you? Who acts?)
2.  **O**bjective (What is the task?)
3.  **S**tyle (Formal? Funny?)
4.  **T**one (Empathetic? Direct?)
5.  **A**udience (Who is reading this?)
6.  **R**esponse Format (JSON? List?)

Let's compare a **Lazy Prompt** vs a **CO-STAR Prompt**.

In [6]:
# The Task: Reject a candidate for a job.
task = "Write a rejection email to a candidate."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)

--- LAZY PROMPT ---
Here are a few options for a rejection email, ranging from a standard, polite version to one that offers a bit more encouragement. Choose the one that best fits your company's culture and the stage of the hiring process.

---

**Option 1: Standard & Polite (Most Common)**

**Subject: Update on Your Application for [Job Title] at [Company Name]**

Dear [Candidate Name],

Thank you for your interest in the [Job Title] position at [Company Name] and for taking the time to [interview with our team / submit your application].

We appreciate you sharing your qualifications and experience with us. We received a large number of applications for this role, and after careful consideration, we have decided to move forward with other candidates whose qualifications and experience were a closer match for the specific requirements of this role at this time.

This was a very competitive search, and we truly appreciate the time and effort you invested in the process.

We wish you t

## 3. Hallucination vs. Creativity

Did the model make up a reason?
Since we didn't give it facts, it **Predicted the most likely reason** (Usually "Experience" or "Volume of applications").

**This is NOT a bug.** It is a feature. The model is *completing the pattern* of a rejection email.

In [7]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'RocketBoots'.

# Objective
Write a rejection email to a candidate named Bob.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)

--- STRUCTURED PROMPT ---
Hi Bob,

Thank you for your interest in RocketBoots.

We appreciate you taking the time to interview. However, the role's requirements have changed significantly since your application. We wish you the best in your job search.

Keep flying.


## 4. Key Takeaway: Ambiguity is the Enemy

Every piece of information you leave out is a gap the model MUST fill with probability.
- If you don't say "Be brief", it picks the most probable length (Avg email length).
- If you don't say "Be rude", it picks the most probable tone (Polite/Neutral).

## Assignment

Write a structured prompt to generate a **Python Function**.
- **Context:** You are a Senior Python Dev.
- **Objective:** Write a function to reverse a string.
- **Constraint:** It must use recursion (no slicing `[::-1]`).
- **Style:** Include detailed docstrings.

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
import os

# Set up your API key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini')

# Structured prompt template
prompt = ChatPromptTemplate.from_template("""
You are a Senior Python Developer with expertise in writing clean, well-documented code.

**Objective:** Write a Python function to reverse a string.

**Constraints:**
- You MUST use recursion to solve this problem
- Do NOT use string slicing (e.g., [::-1])
- Do NOT use any built-in reverse functions

**Style Requirements:**
- Include a comprehensive docstring following PEP 257 conventions
- The docstring must include:
  - A clear description of what the function does
  - Parameters section with type hints
  - Returns section describing the return value and type
  - At least 2 usage examples
- Use descriptive variable names
- Add inline comments explaining the recursive logic

Write only the function code, nothing else.
""")

# Create the chain
chain = prompt | ChatGoogleGenerativeAI(model="gemini-2.5-flash") | StrOutputParser()

# Generate the function
result = chain.invoke({})
print(result)

```python
def reverse_string(input_string: str) -> str:
    """
    Recursively reverses a given string without using slicing or built-in reverse functions.

    This function takes a string and reverses the order of its characters
    by breaking it down into a base case (an empty or single-character string)
    and a recursive step that takes the last character and prepends it
    to the recursively reversed remainder of the string.

    Args:
        input_string (str): The string to be reversed.

    Returns:
        str: The reversed string.

    Examples:
        >>> reverse_string("hello")
        'olleh'

        >>> reverse_string("Python")
        'nohtyP'

        >>> reverse_string("")
        ''

        >>> reverse_string("a")
        'a'
    """
    # Base case: If the string is empty or has only one character,
    # it is already considered reversed, so we return it as is.
    if len(input_string) <= 1:
        return input_string

    # Recursive step:
    # 1. Extract

---



# Unit 2 - Part 2b: Zero-Shot to Few-Shot

## 1. Introduction: In-Context Learning

How does the model learn without training?
This is called **In-Context Learning**.

### The Attention Mechanism (Flowchart)
When you ask a question, the model "looks back" at the previous text to find patterns.

```mermaid
graph TD
    Input[Current Input: 'Angry + Hungry'] -->|Attention Query| History
    subgraph History [The Prompt Examples]
        Ex1[Ex1: Breakfast + Lunch = Brunch]
        Ex2[Ex2: Chill + Relax = Chillax]
    end
    History -->|Pattern Found: Mix words & define| Prediction[Output: Hangry]
```

In [10]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

## 2. Zero-Shot (No Context)

The model relies purely on its training data.

In [12]:
prompt_zero = "Combine 'Angry' and 'Hungry' into a funny new word."
print(f"Zero-Shot: {llm.invoke(prompt_zero).content}")

Zero-Shot: The most common and widely accepted funny word for that combination is:

**Hangry**


## 3. Few-Shot (Pattern Matching)

We provide examples. The Attention Mechanism attends to the **Structure** (`Input -> Output`) and the **Tone** (Sarcasm).

In [13]:
prompt_few = """
Combine words into a funny new word. Give a sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink alcohol before noon)

Input: Chill + Relax
Output: Chillax (What annoying people say when you are panic attacks)

Input: Angry + Hungry
Output:
"""
print(f"Few-Shot: {llm.invoke(prompt_few).content}")

Few-Shot: **Hangry** (The only valid excuse for your sudden transformation into a fire-breathing monster.)


## 4. Critical Analysis

If you provide **bad examples**, the model will learn the **bad pattern**.
This is why Data Quality in your prompt is just as important as code quality.

---



# Unit 2 - Part 2c: Advanced Templates & Theory

## 1. Theory: Engineering vs. Training

### Hard Prompts (Prompt Engineering)
- **What:** You change the text input.
- **Cost:** Cheap, fast, easy to iterate.
- **Use Case:** Prototyping, General tasks.

### Soft Prompts (Fine Tuning)
- **What:** You change the model's internal weights (mathematically).
- **Cost:** Expensive, slow, needs data.
- **Use Case:** Domain specificity (Medical, Legal), Behavioral change.

In [14]:
# Setup
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

## 2. Dynamic Few-Shotting

If you have 1000 examples, you can't fit them all in the context window.
We use a **Selector** to pick the best ones.

### The Selector Flow (Flowchart)
```mermaid
graph LR
    Input[User Input] -->|Semantic Search| Database[Example Database]
    Database -->|Top 3 Matches| Selector
    Selector -->|Inject| Prompt
```

In [15]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. Our Database of Examples
examples = [
    {"input": "The internet is down.", "output": "We are observing connectivity latency."},
    {"input": "This code implies a bug.", "output": "The logic suggests unintended behavior."},
    {"input": "I hate this feature.", "output": "This feature does not align with my preferences."},
]

# 2. Template for ONE example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. The Few-Shot Container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# 4. The Final Chain
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite the input to sound professional."),
    few_shot_prompt,      # Inject examples here
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app sucks."}).content)

This application is not currently delivering an optimal user experience.


## 3. Analysis

Using `FewShotChatMessagePromptTemplate` creates a clean separation between instructions and data. This helps the Attention Mechanism focus on the right things.

In [18]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    {"input": "I love this!", "output": "Positive"},
    {"input": "This is terrible", "output": "Negative"},
    {"input": "It's okay", "output": "Neutral"}
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analyzer."),
    few_shot_prompt,
    ("human", "{text}")
])